In [1]:
import csv
import pandas as pd
import numpy as np
import requests
from IPython.core.display import HTML
from selenium import webdriver
from bs4 import BeautifulSoup
import timeit
import urllib2
import seaborn as sb
import matplotlib.pyplot as plt
import json
%matplotlib inline


In [2]:
###Scraping Data (Takes a couple minutes)
salary = []
company = []
location = []
jobs = []
jobdescription = []
for i in range(31):
    url = "http://www.careerbuilder.com/jobs-data-science?page_number=1&pay=20"
    index = 59
    char = i
    charplus = i + 1
    charplusstring = str(charplus)
    url2 = url[:index] + charplusstring + url[index + 1:]
    rm = requests.get(url2)
    soupm = BeautifulSoup(rm.content, "lxml")
    
    ###Appends job titles
    for d in soupm.findAll(class_='job-title'):
        jobs.append(d.text)
        
    ###Appends job descriptions
    for d in soupm.findAll(class_='job-description show-for-medium-up'):
        jobdescription.append(d.text)
    
    ###Appends salary, company, location.
    z = 0
    for d in soupm.findAll(class_='job-text'):
        if z % 3 == 0:
            salary.append(d.text)
        elif z % 3 == 1:
            company.append(d.text)
        else:
            location.append(d.text)
        
        z += 1
        

In [3]:
###Returning the lists to check
#salary
#company
#location
#jobs
#jobdescription

In [4]:
###Zipping them all together
info = zip(salary, company, location, jobs, jobdescription)

In [5]:
###Putting in DataFrame
df = pd.DataFrame(info)

In [6]:
###Naming Columns
df.columns = ('salary', 'company', 'location', 'title', 'description')

In [7]:
###Breaking location into State/City
df['state'] = [(x[-3:-1]) for x in df['location']]

In [8]:
###Breaking location into City
df['delete'] = df['location'].str.replace('\n', '-------------------------')

In [9]:
df['delete2'] = [(x[-80:-29]) for x in df['delete']]

In [10]:
df['delete3'] = df['delete2'].str.replace('-', '')

In [11]:
df['city'] = df['delete3']

In [12]:
del df['delete']
del df['delete2']
del df['delete3']
del df['location']

In [13]:
###Removing \n from company
df['company'] = df['company'].str.replace('\n', '')

In [14]:
###Removing \n from title
df['title'] = df['title'].str.replace('\n', '')

In [15]:
###Removing \n from description
df['description'] = df['description'].str.replace('\n', '')

In [16]:
###Creating a column for low end of salary
df['low'] = [(x[18:22]) for x in df['salary']]

In [17]:
###Removing '$'
df['low'] = df['low'].str.replace('$', '')

In [18]:
###Removing 'k'
df['low'] = df['low'].str.replace('k', '')

In [19]:
###Removing '-'
df['low'] = df['low'].str.replace('-', '')

In [20]:
###Removing ','
df['low'] = df['low'].str.replace(',', '')

In [21]:
###Creating a column for high end of salary
df['high'] = [(x[-11:-6]) for x in df['salary']]

In [22]:
###Removing '$'
df['high'] = df['high'].str.replace('$', '')

In [23]:
###Removing 'k'
df['high'] = df['high'].str.replace('k', '')

In [24]:
###Removing '-'
df['high'] = df['high'].str.replace('-', '')

In [25]:
###Removing ','
df['high'] = df['high'].str.replace(',', '')

In [26]:
###Fixing '\nFull'
df['high'] = df['high'].str.replace('\nFull', '')

In [27]:
df.dtypes

salary         object
company        object
title          object
description    object
state          object
city           object
low            object
high           object
dtype: object

In [28]:
###START OF CODE TO FIX NANS AND CONVERT TO INTEGERS
dftest = df.copy()

In [29]:
###Turning emptys into nans and dropping
dftest['high'].replace('', np.nan, inplace=True)
dftest['low'].replace('', np.nan, inplace=True)
dftest.dropna(subset=['high'], inplace=True)
dftest.dropna(subset=['low'], inplace=True)

In [30]:
###Turning remaining emptys into 0
dftest['high']=dftest['high'].replace('', '0')
dftest['low']=dftest['low'].replace('', '0', inplace=True)

In [31]:
###Overriding error to return 0s and ints
testing = df['high']

In [32]:
def replace_nulls(x):
    try:
        return int(x)
    except:
        '0'

In [33]:
testing = testing.apply(replace_nulls)

In [34]:
#testing.dropna(inplace=True)

In [35]:
lowtest = df['low']

In [36]:
lowtest = lowtest.apply(replace_nulls)

In [37]:
# lowtest.dropna(inplace=True)

In [38]:
# lowtest

In [39]:
###putting together for a dataframe
salarycolumns = zip(lowtest, testing)

In [40]:
salarycolumnsframe = pd.DataFrame(salarycolumns)

In [41]:
df2 = pd.concat([df, salarycolumnsframe], axis=1)

In [42]:
#Renaming Columns
df2.columns = ('salary', 'company', 'title', 'description', 'state', 'city', 'low', 'high', 'lowsalary', 'highsalary')

In [43]:
###Dropping Columns
del df2['salary']
del df2['low']
del df2['high']

In [44]:
#Getting Rid of Error Salaries
def drop_outliers(x):
    if x > 250:
        return np.nan
    else:
        return x

In [45]:
fixer10 = df2['highsalary']

In [46]:
fixer10 = fixer10.apply(drop_outliers)

In [47]:
#Getting Rid of Error Salaries in the 'Low' Column
def low_outliers(x):
    if x > 5:
        return x
    else:
        return np.nan

In [48]:
lowerfixer = df2['lowsalary']

In [49]:
lowhighfix = zip(lowerfixer, fixer10)

In [50]:
lowhigh = pd.DataFrame(lowhighfix)

In [51]:
df2 = pd.concat([df2, lowhigh], axis=1)

In [52]:
df2['low_salary'] = df2[0]
df2['high_salary'] = df2[1]
#df2.head()

In [53]:
##Dropping Columns
del df2[0]
del df2[1]
del df2['lowsalary']
del df2['highsalary']

In [54]:
#Average Salary
((df2['low_salary'].mean()) + (df2['high_salary'].mean())) / 2

94.93603906062886

In [55]:
###Replacing Nans with Average
def averages(x):
    try:
        return int(x)
    except:
        return 95.0

In [56]:
tester94 = df2['low_salary']

In [57]:
tester94 = tester94.apply(averages)

In [58]:
low_with_average = tester94

In [59]:
###Return the mean for 0.0 cells
def zerofixer(x):
    try:
        return (x / 0) + x
    except:
        return 95.0

In [60]:
low_with_average = low_with_average.apply(zerofixer)

In [61]:
low_testing = tester94

In [62]:
toaddagain = pd.DataFrame(low_testing)

In [63]:
df3 = pd.concat([df2, toaddagain], axis=1)

In [64]:
df4 = df3.copy()

In [65]:
df5 = df2.copy()

In [66]:
###Applying the original 'averages' function directly to the dataframe
df5['low_salary'] = df5['low_salary'].apply(averages)

In [67]:
df5['high_salary'] = df5['high_salary'].apply(averages)

In [68]:
df6 = df5.copy()

In [69]:
###Easier method. Returning the mean if salary is less than $5,000.
def zeros_to_average(x):
    if x < 5:
        return 95.0
    else:
        return x

In [70]:
df5['low_salary'] = df5['low_salary'].apply(zeros_to_average)
df5['average_salary'] = (df5['low_salary'] + df5['high_salary']) / 2

In [71]:
dfreg = df5.copy()

In [72]:
del dfreg['low_salary']
del dfreg['high_salary']

In [73]:
dfreg

,company,title,description,state,city,average_salary
0,Sancus Associates,Associate Data Science,"Associate Data Scientist AdTech $125,000-150,0...",DC,District of Columbia,137.5
1,Analytic Recruiting,"Associate, data Science and market Intelligence","Associate, data Science and market Intelligenc...",NY,New York,100.0
2,Tech Mahindra,BI Data Engineer,Position Title: BI Data Engineer Location: New...,OH,New Albany,95.0
3,Challenge Charter School,6th grade Science & Language Arts Teacher - Mi...,Arizona’s 1st Official Core Knowledge® School;...,AZ,Glendale,41.0
4,CyberCoders,Senior Data Scientist - be the first Data Scie...,This position is open as of 10/18/2016. Senior...,CA,Marina del Rey,175.0
5,Medix,Research Scientist,Research Scientist This Research Scientist wil...,TX,Dallas,60.0
6,BIG WEDNESDAY DIGITAL,Junior Data Scientist - global eCommerce - Ban...,Junior Data Scientist - global eCommerce leade...,MD,Baltimore,97.5
7,Burnett Specialists,Data Scientist,Job Reference #: 41-1014TBD-16CT Data Scientis...,TX,Houston,100.0
8,BIG WEDNESDAY DIGITAL,Senior/Lead/NLP- Data Scientist - Bangkok role...,Senior/Lead/NLP- Data Scientist - Bangkok base...,WA,Seattle,97.5
9,Kelly Scientific Resources,"Sr. Manager, Medical Data Methodology",Kelly Clinical and a highly reputable pharmace...,MA,Cambridge,110.0


In [74]:
dfreg['high_vs_low'] =

SyntaxError: invalid syntax (<ipython-input-74-4db67455f847>, line 1)

In [ ]:
##Bucketing Income
def bucket(x):
    if x['average_salary'] > 95:
        return 1
    else:
        return 0

In [ ]:
dfreg2 = dfreg.copy()

In [ ]:
dfreg2['high_vs_low'] = dfreg2.apply(bucket, axis =1)

In [ ]:
##Now have complete dataset
dfreg2['label'] = dfreg2['high_vs_low']

In [ ]:
del dfreg2['high_vs_low']

In [ ]:
data = dfreg2

<h1> 'Data' is our final dataset </h1>
<br>
<p>
After scraping 30 pages of search results for "Data Science" on careerbuilder.com, cleaning the data, and creating a binary column for 'high' vs 'low' salary we finally have our final dataframe. We determined salaries above the mean of $95,000 would be considered 'high' and anything else would be 'low'.
</p>

In [ ]:
data.head()

In [ ]:
###Testing to see which words in 'Job Titles' correspond to higher salaries

from sklearn.feature_extraction.text import CountVectorizer
v = CountVectorizer(
    binary=True,  
    stop_words='english', 
    max_features=50, 


X = v.fit_transform(data.title).todense()
X = pd.DataFrame(X, columns=v.get_feature_names())
X.head()

In [ ]:
y = data.label

In [ ]:
#Function to help examine the coefficients
def examine_coefficients(model, df9):
    df9 = pd.DataFrame(
        { 'Coefficient' : model.coef_[0] , 'Feature' : df9.columns}
    ).sort_values(by='Coefficient')
    return df9[df9.Coefficient !=0 ]

In [ ]:
###Creates DataFrame of Coefficients for Job Title
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(penalty = 'l1', C = 1.0) # Model with high, little regularization

model.fit(X, y)
datatitle = examine_coefficients(model, X)

In [ ]:
###Testing to see how well the model performed
from sklearn.cross_validation import cross_val_score

for metric in ['accuracy', 'precision', 'recall', 'roc_auc']:
    scores = cross_val_score(model, X, y, scoring=metric)
    print("mean {}: {}, all: {}".format(metric, scores.mean(), scores))
    
# mean roc_auc: 0.829

<h1> Testing the Model </h1>
<br>
<p>
Our model testing words in 'job title' to predict salary had an roc_auc score of 0.74, accuracy of .68, precision of .68, and recall of .59.
</p>
<p> The words in the job title that were the strongest indicators of low salary were 'level', 'coordinator', 'electrical', 'analyst', 'quality', 'supervisor', 'associate', and 'support'.
</p>
<p>
The words in the job title that were the strongest indicators of high salaries were 'architect', 'infrastructure', 'director', 'manager', 'sr', 'senior', 'lead', and 'development'
</p>
<p>
The words in job description that were the strongest indicators of high salaries were 'manager', 'responsible', 'experience', 'senior', 'business',  and 'growing'.
</p>
<p>
The words in job description that were the strongest indicators of low salaries were 'analyst', 'position', 'job', 'support', 'opportunity', 'hire', 'new', 'systems', and 'time'.
</p>

In [ ]:
datatitle

In [ ]:
datadescription

In [ ]:
###Model based on Description
v = CountVectorizer(
    binary=True,  # Create binary features
    stop_words='english', # Ignore common words such as 'the', 'and'
    max_features=50, # Only use the top 50 most common words
)


# This builds a matrix with a row per website (or data point) and column per word (using all words in the dataset)
X = v.fit_transform(data.description).todense()
X = pd.DataFrame(X, columns=v.get_feature_names())
X.head()

In [ ]:
def examine_coefficients2(model, df10):
    df10 = pd.DataFrame(
        { 'Coefficient' : model.coef_[0] , 'Feature' : df10.columns}
    ).sort_values(by='Coefficient')
    return df10[df10.Coefficient !=0 ]

In [ ]:
model = LogisticRegression(penalty = 'l1', C = 1.0) # Model with high, little regularization

model.fit(X, y)
datadescription = examine_coefficients(model, X)

In [ ]:
#data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split

from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import confusion_matrix

import numpy as np
import pandas as pd
import patsy

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.grid_search import GridSearchCV

In [ ]:
###We also created a model based on City/State/Title/Description
X = patsy.dmatrix('~C(city) + C(state) +C(company) + C(title) +C(description)' , data)
y = data['label'].values

In [ ]:
pdf = pd.DataFrame(X, columns=X.design_info.column_names)
pdf['Target'] = y
#pdf

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=77)

In [ ]:
lr = LogisticRegression(solver='liblinear')

In [ ]:
lr_model = lr.fit(X_train, y_train)

In [ ]:
lr_ypred = lr_model.predict(X_test)

In [ ]:
# actual = index; predicted = columns
lr_cm = confusion_matrix(y_test, lr_ypred, labels=lr.classes_)
lr_cm = pd.DataFrame(lr_cm, columns=lr.classes_, index=lr.classes_)
lr_cm

In [ ]:
print classification_report(y_test, lr_ypred, labels=lr.classes_)

In [ ]:
lr_l1 = LogisticRegression(C=500, penalty='l1', solver='liblinear')
lr_l1_model = lr_l1.fit(X_train, y_train)

In [ ]:
lr_l1_model = lr_l1.fit(X_train, y_train)

In [ ]:
lr_l1_ypred = lr_l1_model.predict(X_test)

In [ ]:
lr_l1_cm = confusion_matrix(y_test, lr_l1_ypred, labels=lr_l1.classes_)
lr_l1_cm = pd.DataFrame(lr_l1_cm, columns=lr_l1.classes_, index=lr_l1.classes_)
lr_l1_cm

In [ ]:
print classification_report(y_test, lr_l1_ypred, labels=lr_l1.classes_)

<p>
Our model testing based on city, state, title, description, and company had an f1-score of .68 when we did a regular logistic regression and had an f1-score of .68 when we used an L1 with a C of 500.
</p>